In [240]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np

In [242]:
df = pd.read_csv("C:/Users/Abhijeet/Downloads/Sample - Superstore.csv", encoding="latin1")

In [244]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

In [246]:
summary = df.describe()
summary

,Row ID,Postal Code,Sales,Quantity,Discount,Profit
count,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000
mean,4997.500000,55190.379428,229.858001,3.789574,0.156203,28.656896
std,2885.163629,32063.693350,623.245101,2.225110,0.206452,234.260108
min,1.000000,1040.000000,0.444000,1.000000,0.000000,-6599.978000
25%,2499.250000,23223.000000,17.280000,2.000000,0.000000,1.728750
50%,4997.500000,56430.500000,54.490000,3.000000,0.200000,8.666500
75%,7495.750000,90008.000000,209.940000,5.000000,0.200000,29.364000
max,9994.000000,99301.000000,22638.480000,14.000000,0.800000,8399.976000


In [248]:
#Remove duplicates

df.drop_duplicates(inplace=True)

In [250]:
# Handle missing values (drop if few, fill if important)

df.dropna(subset=["Sales", "Profit", "Order Date"], inplace=True)

In [252]:
# Convert Order Date to datetime

df["Order Date"] = pd.to_datetime(df["Order Date"])
df["Ship Date"] = pd.to_datetime(df["Ship Date"])


In [254]:
# numeric columns are correct type

df["Sales"] = pd.to_numeric(df["Sales"], errors="coerce")
df["Profit"] = pd.to_numeric(df["Profit"], errors="coerce")
df["Quantity"] = pd.to_numeric(df["Quantity"], errors="coerce")

In [256]:
# Remove rows with negative sales

df = df[df["Sales"] >= 0]

In [258]:
# Reset index

df.reset_index(drop=True, inplace=True)

In [260]:
print(df.head())

   Row ID        Order ID Order Date  Ship Date       Ship Mode Customer ID  \
0       1  CA-2016-152156 2016-11-08 2016-11-11    Second Class    CG-12520   
1       2  CA-2016-152156 2016-11-08 2016-11-11    Second Class    CG-12520   
2       3  CA-2016-138688 2016-06-12 2016-06-16    Second Class    DV-13045   
3       4  US-2015-108966 2015-10-11 2015-10-18  Standard Class    SO-20335   
4       5  US-2015-108966 2015-10-11 2015-10-18  Standard Class    SO-20335   

     Customer Name    Segment        Country             City  ...  \
0      Claire Gute   Consumer  United States        Henderson  ...   
1      Claire Gute   Consumer  United States        Henderson  ...   
2  Darrin Van Huff  Corporate  United States      Los Angeles  ...   
3   Sean O'Donnell   Consumer  United States  Fort Lauderdale  ...   
4   Sean O'Donnell   Consumer  United States  Fort Lauderdale  ...   

  Postal Code  Region       Product ID         Category Sub-Category  \
0       42420   South  FUR-BO-10

In [262]:
# Descriptive Analysis

In [264]:
# Sales by Category

print("Sales by Category:\n", df.groupby("Category")["Sales"].sum())

Sales by Category:
 Category
Furniture          741999.7953
Office Supplies    719047.0320
Technology         836154.0330
Name: Sales, dtype: float64


In [266]:
# Profit by Region

print("\nProfit by Region:\n", df.groupby("Region")["Profit"].sum())


Profit by Region:
 Region
Central     39706.3625
East        91522.7800
South       46749.4303
West       108418.4489
Name: Profit, dtype: float64


In [268]:
# Overall statistics

print("\nDataset Summary:\n", df.describe())


Dataset Summary:
             Row ID                     Order Date  \
count  9994.000000                           9994   
mean   4997.500000  2016-04-30 00:07:12.259355648   
min       1.000000            2014-01-03 00:00:00   
25%    2499.250000            2015-05-23 00:00:00   
50%    4997.500000            2016-06-26 00:00:00   
75%    7495.750000            2017-05-14 00:00:00   
max    9994.000000            2017-12-30 00:00:00   
std    2885.163629                            NaN   

                           Ship Date   Postal Code         Sales     Quantity  \
count                           9994   9994.000000   9994.000000  9994.000000   
mean   2016-05-03 23:06:58.571142912  55190.379428    229.858001     3.789574   
min              2014-01-07 00:00:00   1040.000000      0.444000     1.000000   
25%              2015-05-27 00:00:00  23223.000000     17.280000     2.000000   
50%              2016-06-29 00:00:00  56430.500000     54.490000     3.000000   
75%              

In [270]:
# Predictive Analysis

In [272]:
# Predict Profit based on Sales
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [274]:
# Convert dates into numeric for ML

df["Date_Ordinal"] = df["Order Date"].map(pd.Timestamp.toordinal)
X = df[["Date_Ordinal"]]
y = df["Sales"]
model = LinearRegression()
model.fit(X, y)


LinearRegression()

In [276]:
# Add predicted sales column

df["Predicted_Sales"] = model.predict(X)

In [278]:
# Interactive Dashboard using dash

import plotly.express as px
from dash import Dash, dcc, html, Input, Output
app = Dash(__name__)

# Layout
app.layout = html.Div([
    html.H1("Superstore Sales Dashboard"),
    dcc.Dropdown(
        id="category-filter",
        options=[{"label": c, "value": c} for c in df["Category"].unique()],value=None,
        placeholder="Select Category"),
    dcc.Graph(id="sales-over-time"),
    dcc.Graph(id="sales-by-region"),
    dcc.Graph(id="sales-vs-profit")
])



# Callback for cross filtering
@app.callback(
    [Output("sales-over-time", "figure"),
     Output("sales-by-region", "figure"),
     Output("sales-vs-profit", "figure")],
    [Input("category-filter", "value")]
)
def update_dashboard(selected_category):
    # Filter if category selected
    dff = df.copy()
    if selected_category:
        dff = dff[dff["Category"] == selected_category]

    # Line Chart: Sales Over Time
    sales_trend = (dff.groupby("Order Date")["Sales"].sum().reset_index())
    fig1 = px.line(sales_trend,
        x="Order Date", y="Sales",
        title="Sales Over Time" )

    # Bar Chart: Sales by Region
    region_sales = (
        dff.groupby("Region")["Sales"]
        .sum()
        .reset_index() )
    fig2 = px.bar(region_sales,
        x="Region", y="Sales",
        title="Sales by Region")

    # Scatter: Sales vs Profit 
    fig3 = px.scatter(dff,
        x="Sales", y="Profit",
        color="Category",
        title="Sales vs Profit" )
    return fig1, fig2, fig3


In [280]:
# Run app
if __name__ == "__main__":
    app.run(debug=True)